In [3]:
import pandas as pd
import numpy as np
import sklearn
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from sklearn.utils import shuffle
import math

In [4]:
df = pd.read_csv('training_set_VU_DM.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 'gross_bookings_usd',
       'booking_bool'])

In [5]:
df2 = df[['srch_id','prop_id','click_bool', 'booking_bool']]

In [6]:
## create test set
testset = df2.loc[df['srch_id']==1][:10]
for i in range(2,10):
    s4 = df2.loc[df['srch_id']==i][:10]
    testset = pd.concat([testset,s4])
    
testset = testset[['srch_id','prop_id','click_bool', 'booking_bool']]
created_solution = testset[['srch_id','prop_id']]

In [7]:
current_srch_id = None
rank = None
dcg = 0 #Discounted Cumulative Gain
for index, row in created_solution.iterrows():
    if row['srch_id'] != current_srch_id:
        rank = 1
    current_srch_id = row['srch_id']
    current_prop_id = row['prop_id']
    
    correct_value = df2.loc[(df2['srch_id'] == current_srch_id) & (df2['prop_id'] == current_prop_id)]
    if len(correct_value) > 0:
        if correct_value['booking_bool'].iloc[0] == 1:
            dcg_row = 5 / math.log(rank+1,2)
            dcg = dcg + dcg_row
        elif correct_value['click_bool'].iloc[0] == 1:
            dcg_row = 1 / math.log(rank+1,2)
            dcg = dcg + dcg_row
    rank = rank + 1

In [8]:
## testset op goeie volgorde krijgen
testset_volgorde = testset.sort_values(['srch_id','booking_bool', 'click_bool'], ascending=[True, False, False])

In [48]:
def getDCG(solution):
    current_srch_id = None
    rank = None
    dcg = 0#Discounted Cumulative Gain
    for index, row in solution.iterrows():
        if row['srch_id'] != current_srch_id:
            rank = 1
        current_srch_id = row['srch_id']
        current_prop_id = row['prop_id']
    
        correct_value = df2.loc[(df2['srch_id'] == current_srch_id) & (df2['prop_id'] == current_prop_id)]
        if len(correct_value) > 0:
            if correct_value['booking_bool'].iloc[0] == 1:
                dcg_row = 5 / math.log(rank+1,2)
                dcg = dcg + dcg_row
            elif correct_value['click_bool'].iloc[0] == 1:
                dcg_row = 1 / math.log(rank+1,2)
                dcg = dcg + dcg_row
        rank = rank + 1
    return dcg

In [10]:
dcg = getDCG(created_solution)
idcg = getDCG(testset_volgorde[['srch_id','prop_id']])
ndcg = dcg/idcg

In [11]:
ndcg

0.43067655807339306

Simple benchmark

In [53]:
searchids = testset.srch_id.unique()
testset_zonder = testset[['srch_id','prop_id']]
outcome_benchmark = pd.DataFrame()


for sid in searchids:
    data_srch_id = testset_zonder.loc[(testset['srch_id'] == sid)]
    sample_of_srch_id = data_srch_id.sample(5)
    outcome_benchmark = pd.concat([newDF,sample_of_srch_id])


In [54]:
dcg_b = getDCG(outcome_benchmark)
idcg_b = getDCG(testset_volgorde[['srch_id','prop_id']])
ndcg_b = dcg_b/idcg_b
ndcg_b

0.4005149978319906